**LearnPy: Python for Data Analytics**

- Author : Team Algoritma
- Developed by Algoritma's product division and instructors team
___

# Learn Python - Introduction to Machine Learning 1: Classification & Regression

Selamat datang di Learn Python - Introduction to Machine Learning 1. Notebook ini berisi latihan bagaimana cara membuat model untuk klasifikasi dan regresi yang dapat digunakan untuk melatih kemampuan Anda. Pada bagian ini Anda akan diperkenalkan dengan workflow dari Machine Learning mulai dari melakukan *exploratory data analysis* (EDA), melakukan pembersihan data (*preprocessing*), membuat model, dan melakukan evaluasi. Mari berlatih!

##  Machine Learning Workflow: Classification

### Case Study: Coronary Heart Disease

<p align="center">
<img src="assets/chd.jpg" alt="Coronary Heart Disease" width="350">
</p>

Penyakit jantung koroner adalah jenis penyakit jantung ketika arteri jantung tidak dapat mengalirkan cukup darah yang kaya oksigen ke jantung. Kondisi ini mempengaruhi arteri koroner yang lebih besar pada permukaan jantung. Masalah gangguan jantung ini dapat menimbulkan sejumlah keluhan. Mulai dari nyeri dada, sesak napas dan gejala serangan jantung. 

Beberapa faktor yang meningkatkan risiko seseorang terkena jantung koroner diantaranya yaitu:
> - Tekanan darah tinggi
> - Kolesterol dan trigliserida tinggi
> - Diabetes
> - Kegemukan
> - Kebiasaan merokok
> - Peradangan pada pembuluh darah
> - Dan faktor lainnya

Untuk dapat membuat model machine learning yang dapat memprediksi seseorang terkena jantung koroner atau tidak, kita membutuhkan sebuah program/algoritma. Salah satu library yang akan banyak digunakan untuk kebutuhan machine learning baik algoritma atau dataset di Python adalah `sklearn`. Dokumentasinya ada [di sini](https://scikit-learn.org/stable/). Namun sebelum itu, kita harus melakukan load dataset dan melakukan proses pembersihan data terlebih dahulu.

### 1. Load Data: Coronary Heart Disease Dataset

Mari kita mulai dengan mengimpor library yang diperlukan.

In [ ]:
# library untuk manipulasi data
import pandas as pd
import numpy as np

# library untuk menampilkan visualisasi
import matplotlib.pyplot as plt

# library untuk preprocessing -> scaling data
from sklearn.preprocessing import StandardScaler

# library untuk preprocessing -> membagi dataset menjadi train dan test set
from sklearn.model_selection import train_test_split

# library untuk membuat model machine learning
from sklearn import svm

# library untuk evaluasi model
from sklearn import metrics

Mari kita baca dataset `data_input/CHDdata.csv`

In [ ]:
# load dataset
patient = pd.read_csv("data_input/CHDdata.csv")
patient.head()

 🔎 **Penjelasan Data**

_variabel prediktor:_
- `sbp` : _Systolic blood pressure_ atau tekanan darah sistolik (mmHg).
- `tobacco`: Konsumsi tembakau per tahun (kg).
- `ldl`: _Low density lipoprotein_ atau kadar kolesterol LDL dalam darah.
- `adiposity`: Adipositas atau kelebihan timbunan lemak dalam tubuh.
- `famhist`: _Family history_ atau riwayat keluarga jantung koroner (present = ada riwayat, absent = tidak ada riwayat).
- `typea`: Skor kepribadian tipe A. Kepribadian tipe A memiliki tingkat stress yang lebih tinggi.
- `obesity`: Skor _body mass index_ (BMI).
- `alcohol`: Konsumsi alkohol.
- `age`: Usia pasien (tahun).

_variabel target:_

- `chd`: _Coronary heart disease_ (0: sehat, 1: jantung koroner)

<div class="alert alert-info">
<a href="https://www.kaggle.com/datasets/billbasener/coronary-heart-disease"><b>[Coronary Heart Disease Dataset]</b></a> adalah data yang mengandung informasi medis dari pasien. Tujuan dari projek ini adalah <b>membantu tenaga medis dengan memprediksi apakah seorang pasien terkena jantung koroner atau tidak </b> berdasarkan data medis dan gaya hidup pasien.
</div>

### 2. Exploratory Data Analysis (EDA)

Tahapan-tahapan yang dapat dilakukan pada EDA antara lain:

> - Melihat tipe data, missing value dan data duplikat
> - Melihat deskripsi statistik data

#### 2.1 Tipe Data, Missing Value dan Duplikat

##### `1️⃣` Tipe Data

Tipe data yang tepat sangat penting sebelum membangun model machine learning. **Kesalahan tipe data dapat menyebabkan misinterpretasi oleh model dan menurunkan performanya.**

Contohnya, jika kolom numerik masih dalam bentuk string, model tidak dapat melakukan operasi matematika dan menghasilkan error.

Mari kita melihat dan menyesuaikan tipe data dari dataset kita.

In [ ]:
# melihat tipe data tiap kolom
patient.dtypes

> Terlihat bahwa prediktor `famhist` memiliki tipe data `object`. Kita perlu mengubahnya menjadi data `category` menggunakan fungsi `.astype()` yang sudah disediakan oleh python.

#### Quiz 1

___
1. Ubah kolom bertipe object menjadi category.

> ✨ Hint: Perhatikan gambar berikut!

<!-- 
Reference answer :

```python
# list kolom yang akan dijadikan kategori
cat_features = ['famhist']

# mengubah tipe data
patient[cat_features] = patient[cat_features].astype('category')

# menampilkan tipe data
patient.dtypes
```
-->


![](assets/svm_data_type.png) 

In [ ]:
# your code here


##### `2️⃣` Missing Value

Missing value dapat terjadi karena berbagai alasan. Hal ini dapat memengaruhi performa model atau bahkan menggagalkan proses pemodelan. Oleh karena itu, penting untuk menangani missing value sebelum membangun model.

 🔎 **Menangani Missing Value**

Untuk treatment missing values sebenarnya ada beberapa cara umum yang bisa digunakan. Antara lain:

> - **Imputasi:** Mengisi nilai yang hilang dengan nilai lain: Menggunakan metode `.fillna()`
> - **Drop:** Menghapus observasi yang memiliki nilai missing: Menggunakan metode `.dropna()`. Namun jika nilai missing > 5%, maka tidak disarankan menggunakan metode ini.

Metode imputasi bisa berbeda tergantung jenis tipe datanya. Berikut detail metode imputasi yang bisa digunakan.

**Untuk kolom categorical:**

> - Isi menggunakan nilai lainnya seperti `other` atau `missing`
> - Isi menggunakan nilai terbanyak yg muncul (`mode`)

**Untuk kolom numerical:**

> - Isi menggunakan nilai tengah seperti `mean` atau `median`
> - Isi dengan nilai 0

**Untuk kolom datetime:**

> - Menggunakan metode `bfill` : melakukan imputasi dari bawah ke atas
> - Menggunakan metode `ffill` : melakukan imputasi dari atas ke bawah

___
2. Ada berapa nilai missing pada kolom `famhist` ?

> ✨ Hint: Gunakan tambahan fungsi **.sum()** untuk melihat jumlah angkanya.

- [ ] Tidak ada
- [ ] 2
- [ ] 10

<!-- 
Reference answer :

```python
# menampilkan missing value tiap kolom
patient.isna().sum()
```

Berdasarkan fungsi di atas, dataset patient tidak memiliki nilai missing. Sehingga pada kolom famhist juga TIDAK ADA missing value.

-->

In [ ]:
# your code here


##### `3️⃣` Duplicate Value

Nilai duplikat adalah baris yang memiliki kesamaan nilai dengan 1 atau lebih baris yang lain.

**Nilai duplikat dalam dataset dapat menyebabkan hasil pemodelan yang bias dan tidak akurat karena mementingkan nilai-nilai yang duplikat.**

Oleh karena itu, penting untuk mengidentifikasi nilai duplikat sebelum membangun model machine learning.

___
3. Tampilkan berapa jumlah baris yang duplikat pada dataset.

> ✨ Hint: Gunakan tambahan fungsi **.sum()** untuk melihat jumlah angkanya.

- [ ] Tidak ada
- [ ] 2
- [ ] 10

<!-- 
Reference answer :

```python
# menampilkan data duplikat
patient.duplicated().sum()
```

Berdasarkan fungsi di atas, dataset patient tidak memiliki nilai duplikat.

-->



In [ ]:
# your code here


#### 2.2 Melihat Deskripsi Statistik Data 

___
4. Menggunakan dataframe `patient`, berapa nilai tengah tekanan darah sistolik pasien? 

> Hint : Anda mungkin membutuhkan method `describe()` untuk melihat statistik data.

- [ ] 124 mmHg
- [ ] 134 mmHg
- [ ] 138 mmHg

<!-- 

Reference answer :

```python
# melihat statistik dataset
patient.describe()
```

Jadi nilai tengah (median) dari tekanan darah sistolik pasien sebesar 134 mmHg => hipertensi tingkat 1

-->


In [ ]:
# your code here


___
5. Menggunakan data pada nomor 1, berapa skor rata-rata _Body Mass Index_ pasien?

- [ ] 25 Kg/M²
- [ ] 26 Kg/M²
- [ ] 28 Kg/M²

<!-- 

Berdasarkan data statistik pada nomor 1, rata-rata pasien memiliki skor BMI (obesity) sebesar 26 Kg/M²a

-->

___
6. Menggunakan data pada nomor 1, berapa jumlah pasien yang memiliki riwayat keturunan penyakit jantung?

> Hint : Anda mungkin membutuhkan method `include()` untuk melihat statistik data pada data kategorik.

- [ ] 192 pasien
- [ ] 270 pasien
- [ ] 462 pasien

<!-- 

Reference answer :

```python
# menampilkan data statistik untuk tipe data kategorikal
patient.describe(include='category')
```

Dari 462 pasien, 270 pasien tidak memiliki riwayat (absent). Jadi pasien yang memiliki riwayat keturunan penyakit jantung sebesar 462 - 270 = 192 pasien.

-->

In [ ]:
# your code here


### 3. Data Preprocessing

Data preprocessing merupakan proses untuk membersihkan data dari data mentah menjadi data yang baik dan lengkap untuk proses modeling. Data yang bersih dan lengkap akan menghasilkan model machine learning yang efektif dan efisien serta memiliki performa yang lebih baik.

Proses ini bertujuan untuk mengurangi waktu komputasi dan resource yang dibutuhkan saat proses pembuatan model dan meningkatkan performa model machine learning.

Tahapan-tahapan yang dapat dilakukan pada data preprocessing antara lain:

> - Menangani data outliers
> - Menangani data kategorikal
> - Membagi dataset menjadi data train dan test
> - Melakukan scaling data

#### 3.1 Handling Numerical Data: Outliers

Outlier adalah observasi dengan nilai yang jauh berbeda dari mayoritas data. Hal ini dapat disebabkan oleh kesalahan input, anomali, atau kesalahan perhitungan. Menangani outlier penting dalam pemodelan machine learning karena dapat memengaruhi performa model.

 🔎 **Menangani Outliers**
 
Ada beberapa cara untuk menangani outliers:
1. **Remove outlier**: Menghilangkan dan menghapus keseluruhan outlier. Metode ini tidak disarankan jika jumlah outliers > 5% dari total data.
2. **Imputasi**: Mengganti nilai outliers dengan nilai yang diperbolehkan (sesuai kebutuhan).
3. ***Scaling***: Teknik untuk mentrasformasi sehingga nilai-nilai outlier ini tidak akan berpengaruh besar terhadap hasil analisis. Teknik scaling bermacam-macam, seperti standarisasi, normalisasi, min-max scaler dan lain sebagainya.


Mari kita identifikasi nilai outlier pada data kita dengan menggunakan boxplot:

In [ ]:
# list kolom numerik
num_features = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

# visualisasi boxplot
patient.boxplot(column=num_features)
plt.show()

> 💡 **INSIGHT:** Dari visualisasi boxplot di atas terlihat adanya outlier pada beberapa kolom seperti `sbp`, `tobacco`, `ldl`, `typea`, `obesity`, dan `alcohol`. Untuk mengatasi ini, kita coba menggunakan teknik ketiga yaitu scaling. Hal ini dikarenakan cukup banyak outlier dan menghindari kehilangan banyak data jika dihapus. Sehingga akan digunakan proses scaling untuk mengurangi pengaruh dari outlier. Proses scaling akan dilakukan setelah train-test splitting.

#### 3.2 Handling Categorical Data

Model machine learning hanya bisa belajar dari data berupa angka. Jadi jika ada prediktor yang memilili tipe data bukan angka, perlu diubah terlebih dahulu ke bentuk angka. 

Prediktor `famhist` merupakan data kategorik dan termasuk tipe nominal karena tidak memiliki hubungan bertingkat antar kategorinya. Untuk itu akan digunakan metode *dummy variable encoding* membuat kolom baru bertipe biner untuk masing-masing kategorinya.

Untuk melakukan **Dummy Variable Encoding** kita menggunakan `pd.get_dummies()` dengan memasukkan kolom yang ingin kita ubah. Adapun parameter antara lain:

- `data`: data yang ingin diubah menjadi numerikal

- `columns`: list nama kolom yang akan dilakukan dummy variable encoding

- `drop_first`: Drop salah satu kolom (kolom pertama). Default False (Tidak ada kolom di drop). True agar kolom hasil dummies tidak rendundan.

- `dtype` : Mengubah tipe data hasil encoding menjadi suatu tipe data

Dokumentasi `pd.get_dummies()` dapat diakses [di sini](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

Mari kita coba menerapkan **Dummy Variable Encoding** untuk kolom yang telah kita disepakati

#### Quiz 2

1. Menggunakan fungsi **.get_dummies()**, ubah kolom kategorical `famhist` menjadi angka numerik bertipe `int64`. Atur parameter **drop_first** agar bernilai True.

> ✨ Hint: Perhatikan gambar di bawah ini.

<!-- 
Reference answer :

```python
# list prediktor yang memiliki tipe nominal
nominal_features = ['famhist']

# implementasi pd.get_dummies
patient = pd.get_dummies(patient, 
                         columns = nominal_features, 
                         drop_first = True, 
                         dtype = 'int64')
patient.head()
```
-->

![](assets/svm_categorical_data.png) 

In [ ]:
# your code here


#### 3.3 Train-Test Splitting (*Cross Validation*)

*Cross Validation* digunakan untuk mengetahui kemampuan model memprediksi data baru.

Untuk memahami performa model, membagi dataset menjadi training data dan testing data adalah strategi yang baik dengan menggunakan fungsi `train_test_split()`

Pisahkan dataset dengan menggunakan fungsi `train_test_split()`. Anda harus melewati 3 parameter `features`, `target`, dan `test_size`. Selain itu, Anda dapat menggunakan `random_state` untuk memilih record secara acak.

Kita dapat menggunakan `train_test_split` dengan beberapa parameter sebagai berikut.
- `*arrays`: dataframe yang kita gunakan (dipisah , untuk yang prediktor dan target variable)
- `test_size`: jumlah persentase dari data yang akan digunakan sebagai data test
- `train_size`: jumlah persentase dari data yang akan digunakan sebagai data test (akan otomatis terisi jika `test_size` diberi nilai)
- `random_state`: nilai random number generator (RNG). Jika kita memasukkan suatu nilai integer untuk parameter ini maka akan menghasilkan hasil yang sama untuk nilai yang sama. Jika kita mengubah nilainya, maka hasilnya akan berbeda.

> **💡 NOTES**: Biasanya data dibagi menjadi 80:20 atau 70:30 (train size:test size). Porsi yang besar selalu digunakan untuk training

In [ ]:
# memilih kolom target
target = 'chd'

# menyimpan data kolom target ke dalam variabel y
y = patient[target]

# menghapus kolom target sehingga tersisa kolom prediktor dan dimasukkan ke variabel X
X = patient.drop(columns=[target])

___
2. Menggunakan fungsi **train_test_split()**, bagi dataset menjadi data train dan data test dengan proporsi 70% data train dan 30% data test. Atur parameter `random_state` menjadi 100.

<!-- 
Reference answer :

```python
# split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.30, # 70% training and 30% test
                                                    random_state = 100)
```
-->


In [ ]:
# your code here


___
3. Berdasarkan hasil `train_test_split`, berapa baris data yang digunakan untuk melatih model?

- [ ] 70 data
- [ ] 139 data
- [ ] 323 data

<!-- 

Reference answer :

```python
# cek dimensi data train
X_train.shape
```

Dari 462 data dibagi menadi 70% data train (323 data) dan 30% data test (139 data).

-->

In [ ]:
# your code here


___
4. Berdasarkan hasil `train_test_split`, berapa baris data yang digunakan untuk menguji model?

- [ ] 30 data
- [ ] 139 data
- [ ] 323 data

<!-- 

Reference answer :

```python
# cek dimensi data test
X_test.shape
```

Dari 462 data dibagi menadi 70% data train (323 data) dan 30% data test (139 data).

-->

In [ ]:
# your code here


#### 3.4 Scaling (*Z Score Nomalization*)

Proses scaling dilakukan untuk menyeragamkan skala data antar prediktor. Hal ini penting dilakukan karena jika skala data antar prediktor tidak seragam, maka model machine learning akan bias (lebih memperhatikan) prediktor yang memiliki skala yang lebih besar. Hal ini akan membuat prediktor lain yang memiliki skala lebih kecil seakan-akan tidak memiliki pengaruh terhadap model dan akan membuat performa model tidak maksimal.

Metode scaling sangat banyak sekali seperti standarisasi dan normalisasi. Untuk kasus ini kita akan menggunakan metode standarisasi Z score menggunakan fungsi `StandardScaler` dari library `sklearn` yang membuat data kita memiliki nilai `mean = 0` dan `standar deviasi = 1`.

In [ ]:
# membuat object dari class StandardScaler()
scaler = StandardScaler()

# proses fit scaling
scaler.fit(X_train[num_features])

___
5. Setelah proses fit scaling data train pada kolom numerik yang terkumpul pada list `num_features`, lakukan proses transformasi pada kolom numerik yang sama untuk **data train** dan **data test** menggunakan fungsi **scaler.transform()**.

<!-- 
Reference answer :

```python
# proses transform scaling
X_train[num_features] = scaler.transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])
```
-->


In [ ]:
# your code here


In [ ]:
# melihat data setelah proses scaling
X_train.head()

> **💡 NOTES**: Proses fit scaling hanya dilakukan pada data train untuk mencegah kebocoran (leak) pada data test. Hal ini dilakukan agar informasi pada data test benar-benar tidak bocor/tidak diketahui saat pembuatan model sehingga hasil evaluasi model lebih dapat dipertanggung jawabkan.

### 4. Model Building: Support Vector Classifier (SVC)

Untuk mesin dapat belajar sesuai dengan kebutuhan kita, kita memerlukan algoritma yang disebut model. Terdapat banyak jenis model **machine learning** dan salah satu contohnya adalah Support Vector Machine.

SVM dapat memecahkan masalah klasifikasi menggunakan `Support Vector Classifier (SVC)` dan regresi menggunakan `Support Vector Regressor (SVR)` dan dapat diperluas untuk memodelkan non-linearitas dalam data.

#### 4.1 SVC Implementation

🔻Setelah semua proses persiapan data selesai, kini adalah saatnya untuk 'mesin' belajar dan diuji oleh data kita.

Langkah-langkah membangun model SVM. 
> 1. Import library SVC dari SVM.
> 2. Buat variabel `model_svc` untuk menyimpan hasil model `SVC()`.

#### Quiz 3

1. Setelah proses preprocessing selesai, saatnya membuat model machine learning menggunakan algoritma SVM. Silakan panggil fungsi **SVC()** dari library **svm**, lalu masukkan ke variabel `model_svc`. Kemudian lakukan proses training menggunakan fungsi **.fit()**.

> ✨ Hint: Masukkan data prediktor dan target di dalam fungsi **.fit()**.

<!-- 
Reference answer :

```python
# membuat object SVC dari class svm
model_svc = svm.SVC()

# proses training
model_svc.fit(X_train, y_train)
```
-->


In [ ]:
# your code here


#### 4.2 Model Prediction

Setelah model dilatih, model siap digunakan untuk memprediksi data baru. Model akan memprediksi data test yang telah kita siapkan sebelumnya.

___
2. Menggunakan model yang telah dilatih, silakan gunakan model tersebut untuk memprediksi **data train** lalu masukkan hasilnya ke dalam variabel `y_pred_train`.

> ✨ Hint: Gunakan prediktor dari data train.

<!-- 
Reference answer :

```python
# prediksi data train
y_pred_train = model_svc.predict(X_train)
```
-->


In [ ]:
# your code here


In [ ]:
# menampilkan 10 data hasil prediksi
y_pred_train[:10]

___
3. Menggunakan model yang telah dilatih, silakan gunakan model tersebut untuk memprediksi **data test** lalu masukkan hasilnya ke dalam variabel `y_pred_test`.

> ✨ Hint: Gunakan prediktor dari data test.

<!-- 
Reference answer :

```python
# prediksi data test
y_pred = model_svc.predict(X_test)
```
-->


In [ ]:
# your code here


In [ ]:
# menampilkan 10 data hasil prediksi
y_pred_test[:10]

**💡NOTES**: Walaupun kita sudah berhasil memprediksi, kita tetap perlu memastikan bahwa hasil prediksinya banyak yang benar / sudah sesuai dengan melakukan model evaluation

### 5. Model Evaluation

Setelah kita melakukan prediksi akan dilakukan evaluasi model dengan membandingkan nilai aktual dan hasil prediksi. Perbedaan antara nilai prediksi dengan nilai aktual disebut sebagai *error*, semakin kecil *error* maka semakin bagus model kita, karena nilai prediksi semakin mendekati nilai aslinya.

Salah satu cara yang sering digunakan dalam menghitung error pada kasus klasifikasi adalah nilai akurasi. Berikut adalah formula menghitung nilai akurasi:

$$Accuracy = {\text{Jumlah prediksi benar} \over \text{Total prediksi}}.$$

> Misal, saya punya 100 data, kemudian model berhasil memprediksi dengan benar 90 data, maka akurasinya adalah
> 90/100 = 90%

🔻 Fungsi `metrics.accuracy_score()` dari library `sklearn` berfungsi untuk menghitung akurasi berdasarkan data yang telah diprediksi oleh model. Parameter yang dimasukkan adalah:
* `y_true`: target yang sebenarnya (ground truth) $\rightarrow$ diambil dari `y_test`
* `y_pred`: target hasil prediksi model

Dokumentasi metrik evaluasi `accuracy` dan metrik lainnya dapat diakses [di sini](https://scikit-learn.org/stable/api/sklearn.metrics.html).

> Untuk mengetahui apakah model yang Anda buat underfit, good fit, atau overfit, mari bandingkan peforma model ketika memprediksi data train dan data test.

Evaluasi model menggunakan data train:

In [ ]:
# evaluasi model
accuracy_train = metrics.accuracy_score(y_train, y_pred_train)

# tampilkan hasil evaluasi
print(f"Accuracy Train : {accuracy_train:,.2f}")

Evaluasi model menggunakan data test:

In [ ]:
# evaluasi model
accuracy_test = metrics.accuracy_score(y_test, y_pred_test)

# tampilkan hasil evaluasi
print(f"Accuracy Test : {accuracy_test:,.2f}")

Membandingkan evaluasi model pada data train dan data test:

In [ ]:
# menampilkan perbandingan evaluasi model pada data train dan data test
pd.DataFrame(data = {"Accuracy": [accuracy_train, accuracy_test]},
             index = ["Data Train", 'Data Test'])

> 💡 **INSIGHT:** Dari hasil evaluasi, dapat dikatakan bahwa model kita belum memiliki performa yang baik karena mengalami overfitting. Hal ini ditandai dengan nilai akurasi pada data train dan test berbeda jauh. Untuk memperbaiki performa, perlu kita lakukan proses tuning hyperparameter.

### 6. Model Improvement: Tuning Hyperparameter SVC (Opsional)

Model tuning berguna untuk memperbaiki model kita dengan cara merubah nilai pada hyperparameter. Namun proses tuning tidak selalu menghasilkan model yang lebih baik sehingga kita perlu lakukan *trial and error* untuk mencari hyperparameter yang mampu meningkatkan performa model.

Adapun *hyperparameter* yang sering digunakan dalam model SVC antara lain:
- `C (regularization) [default = 1, range = 0.0 - inf]`: Regularisasi mengontrol margin dan jumlah kesalahan klasifikasi, semakin besar nilai C maka semakin kecil jumlah kesalahan, semakin kecil nilai C, semakin besar nilai kesalahan yang ditoleransi.
- `kernel [default = 'rbf', option = ['rbf', 'poly', 'linear', 'sigmoid']`: Mempengaruhi bagaimana pemisah (*hyperplane*) terbentuk, untuk data yang dapat dipisahkan secara garis linear (garis lurus) dapat menggunakan 'linear'. Untuk data-data non-linear dapat menggunakan kernel `rbf`.
*Note: Kebanyakan data berbentuk non-linear, sehingga sebaiknya selalu memilih kernel `rbf`*
- `gamma [default = 'scale', option = ['scale', 'auto', positive float number]`: Semakin besar nilai gamma, garis pemisah semakin mengikuti bentuk data (semakin kompleks), semakin kecil, maka semakin garis pemisah lebih general (lebih sederhana).

Untuk hyperparameter dan penjelasan lebih detail dapat dilihat pada [dokumentasi SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

Mari kita coba untuk tuning model Klasifikasi kita dengan nama `model_svc_tuning`

Buatlah model baru bernama `model_svc_tuning` yang memiliki parameter C=100, kernel=rbf, dan gamma=0.001. Lalu lakukan proses **.fit()** pada model tersebut menggunakan data train.

> ✨ Hint: Gunakan prediktor dari data train.

<!-- 
Reference answer :

```python
# membuat model svc tuning
model_svc_tuning = svm.SVC(C = 100, kernel = 'rbf', gamma = 0.001)

# melakukan pelatihan ulang
model_svc_tuning.fit(X_train, y_train)
```
-->


In [ ]:
# your code here


Mari kita lihat performa model hasil tuning untuk memprediksi data train.

In [ ]:
# prediksi data train menggunakan model hasil tuning
y_pred_train_tuning = model_svc_tuning.predict(X_train)

# evaluasi model
accuracy_train_tuning = metrics.accuracy_score(y_train, y_pred_train_tuning)

Mari kita lihat performa model hasil tuning untuk memprediksi data test.

In [ ]:
# prediksi data test menggunakan model hasil tuning
y_pred_test_tuning = model_svc_tuning.predict(X_test)

# evaluasi model
accuracy_test_tuning = metrics.accuracy_score(y_test, y_pred_test_tuning)

Mari kita lihat perbandingan performa model data data train dan test sebelum dan sesudah tuning.

In [ ]:
# Menampilkan perbandingan evaluasi model hasil tuning pada data train dan data test
pd.DataFrame(data = {"Accuracy Train": [accuracy_train, accuracy_test],
                     "Accuracy Test": [accuracy_train_tuning, accuracy_test_tuning]},
             index = ["Base Model", 'Tuning Model'])

**✨ INSIGHT ✨**
> Setelah dilakukan proses tuning, skor akurasi pada data test naik menjadi 71%. Hal ini menandakan bahwa model hasil tuning sudah bisa dikatakan model yang lebih baik. Meskipun model nya mengalami **Overfitting** karena nilai akurasi data train dan test tidak sama, tetapi hal ini masih bisa ditoleransi karena perbedaannya < 10%.

### **Notes**

> **Reference anwer dapat Anda lihat pada setiap markdown soal**

##  Machine Learning Workflow: Regression

### Case Study: Medical Insurance

<p align="center">
<img src="assets/insurance.png" alt="Medical Insurance" width="400">
</p>

Asuransi medis atau sering disebut juga asuransi kesehatan adalah jenis asuransi yang memberikan perlindungan finansial terhadap biaya yang terkait dengan perawatan kesehatan. Sama seperti jenis asuransi lainnya, asuransi medis juga membutuhkan pembayaran untuk setiap waktu tertentu. Besarnya biaya yang ditagih oleh pihak asuransi tergantung pada beberapa faktor seperti kondisi medis, jumlah anak, wilayah tempat tinggal, dan lainnya.

Pada studi kasus kali ini, akan diberikan dataset biaya tagihan pembayaran asuransi medis di wilayah Amerika Serikat. Tugas kita adalah memprediksi berapa tagihan yang musti dibayarkan berdasarkan beberapa faktor yang ada.

### 1. Load Data: Medical Insurance Dataset

Mari kita mulai dengan mengimpor library yang diperlukan.

In [ ]:
# library untuk manipulasi data
import pandas as pd
import numpy as np

# library untuk menampilkan visualisasi
import matplotlib.pyplot as plt

# library untuk preprocessing -> scaling data
from sklearn.preprocessing import StandardScaler

# library untuk preprocessing -> membagi dataset menjadi train dan test set
from sklearn.model_selection import train_test_split

# library untuk membuat model machine learning
from sklearn import svm

# library untuk evaluasi model
from sklearn import metrics

Mari kita baca dataset `data_input/insurance.csv`

In [ ]:
# load dataset
insurance = pd.read_csv("data_input/insurance.csv")
insurance.head()

 🔎 **Penjelasan Data**

_variabel prediktor:_
- `age` $\rightarrow$ usia peserta asuransi (tahun).
- `sex` $\rightarrow$ jenis kelamin (male/female).
- `bmi` $\rightarrow$ _Body Mass Index_ atau masa index tubuh (kg/m2).
- `children` $\rightarrow$ jumlah anak yang ditanggung asuransi.
- `smoker` $\rightarrow$ apakah peserta asuransi merokok? (yes/no).
- `region` $\rightarrow$ wilayah tempat peserta asuransi tinggal (northeast, southeast, southwest, dan northwest)

_variabel target:_

- `charges` $\rightarrow$ biaya tagihan asuransi

<div class="alert alert-info">
<a href="https://www.kaggle.com/datasets/mirichoi0218/insurance/data"><b>[Medical Insurance Dataset]</b></a> adalah data yang mengandung informasi peserta asuransi medis. Tujuan dari projek ini adalah <b>untuk memprediksi besaran tagihan yang musti dibayarkan kepada pihak asuransi </b> berdasarkan data-data peserta.
</div>

### 2. Exploratory Data Analysis (EDA)

Tahapan-tahapan yang dapat dilakukan pada EDA antara lain:

> - Melihat tipe data, missing value dan data duplikat
> - Melihat deskripsi statistik data

#### 2.1 Tipe Data, Missing Value dan Duplikat

##### `1️⃣` Tipe Data

Tipe data yang tepat sangat penting sebelum membangun model machine learning. **Kesalahan tipe data dapat menyebabkan misinterpretasi oleh model dan menurunkan performanya.**

Contohnya, jika kolom numerik masih dalam bentuk string, model tidak dapat melakukan operasi matematika dan menghasilkan error.

Mari kita melihat dan menyesuaikan tipe data dari dataset kita.

In [ ]:
# melihat tipe data tiap kolom
insurance.dtypes

> Terlihat bahwa prediktor `sex`, `smoker`, dan `region` memiliki tipe data `object`. Kita perlu mengubahnya menjadi data `category` menggunakan fungsi `.astype()` yang sudah disediakan oleh python.

In [ ]:
# list kolom yang akan dijadikan kategori
cat_features = ['sex', 'smoker', 'region']

# mengubah tipe data
insurance[cat_features] = insurance[cat_features].astype('category')

# menampilkan tipe data
insurance.dtypes

##### `2️⃣` Missing Value

Missing value dapat terjadi karena berbagai alasan. Hal ini dapat memengaruhi performa model atau bahkan menggagalkan proses pemodelan. Oleh karena itu, penting untuk menangani missing value sebelum membangun model.

 🔎 **Menangani Missing Value**

Untuk treatment missing values sebenarnya ada beberapa cara umum yang bisa digunakan. Antara lain:

> - **Imputasi:** Mengisi nilai yang hilang dengan nilai lain: Menggunakan metode `.fillna()`
> - **Drop:** Menghapus observasi yang memiliki nilai missing: Menggunakan metode `.dropna()`. Namun jika nilai missing > 5%, maka tidak disarankan menggunakan metode ini.

Metode imputasi bisa berbeda tergantung jenis tipe datanya. Berikut detail metode imputasi yang bisa digunakan.

**Untuk kolom categorical:**

> - Isi menggunakan nilai lainnya seperti `other` atau `missing`
> - Isi menggunakan nilai terbanyak yg muncul (`mode`)

**Untuk kolom numerical:**

> - Isi menggunakan nilai tengah seperti `mean` atau `median`
> - Isi dengan nilai 0

**Untuk kolom datetime:**

> - Menggunakan metode `bfill` : melakukan imputasi dari bawah ke atas
> - Menggunakan metode `ffill` : melakukan imputasi dari atas ke bawah

In [ ]:
# menampilkan missing value tiap kolom
insurance.isna().sum()

> Karena tidak ada nilai yang hilang, maka lanjut ke tahap berikutnya.

##### `3️⃣` Duplicate Value

Nilai duplikat adalah baris yang memiliki kesamaan nilai dengan 1 atau lebih baris yang lain. **Nilai duplikat dalam dataset dapat menyebabkan hasil pemodelan yang bias dan tidak akurat karena mementingkan nilai-nilai yang duplikat.** Oleh karena itu, penting untuk mengidentifikasi nilai duplikat sebelum membangun model machine learning.

Untuk melihat jumlah data duplikat pada dataset, kita bisa menggunakan fungsi `.duplicated().sum()`. Sedangkan untuk melihat mana saja yang duplit, bisa menggunakan subsetting.

In [ ]:
# menampilkan jumlah data yang duplikat
insurance.duplicated().sum()

In [ ]:
# melihat data yang duplikat menggunakan subsetting
insurance[insurance.duplicated(keep=False)]

Untuk menghapus data duplikat, bisa menggunakan fungsi `.drop_duplicate()`.

Ada beberapa parameter pada fungsi tersebut, diantaranya yaitu:
- `subset`: identifikasi duplikat berdasarkan kolom tertentu. Default: menggunakan semua kolom.
- `keep`: menentukan data duplikat mana yang akan dipertahankan. Ada 3 pilihan yaitu:
  - `first`: hapus semua duplikat kecuali data pertama (**default**).
  - `last`: hapus semua duplikat kecuali data terakhir.
  - `False`: hapus semua data duplikat.
- `inplace`: apakah ingin langsung mengupdate dataframe?
  - `True`: langsung update dataframe tanpa menyimpan ulang.
  - `False`: tidak langsung update dataframe (**default**).


In [ ]:
# menghapus data duplikat dan mempertahankan data pertama
insurance = insurance.drop_duplicates()
insurance

#### Quiz 1

#### 2.2 Melihat Deskripsi Statistik Data 

___
1. Menggunakan dataframe `insurance`, berapa rata-rata umur peserta asuransi?

> Hint : Anda mungkin membutuhkan method `describe()` untuk melihat statistik data.

- [ ] 39 tahun
- [ ] 51 tahun
- [ ] 64 tahun

<!-- 

Reference answer :

```python
# melihat statistik dataset
insurance.describe()
```

Jadi rata-rata umur peserta asuransi yaitu 39 tahun.

-->


In [ ]:
# your code here


___
2. Menggunakan data pada nomor 1, berapa rata-rata jumlah anak yang didaftarkan asuransi?

- [ ] 1 anak
- [ ] 2 anak
- [ ] 3 anak

<!-- 

Berdasarkan data statistik pada nomor 1, rata-rata jumlah anak yang didaftarkan asuransi yaitu 1.

-->

___
3. Kebanyakan peserta asuransi tinggal di daerah mana?

> Hint : Anda mungkin membutuhkan method `include()` untuk melihat statistik data pada data kategorik.

- [ ] southeast
- [ ] northeast
- [ ] northwest

<!-- 

Reference answer :

```python
# menampilkan data statistik untuk tipe data kategorikal
insurance.describe(include='category')
```

Sejumlah 364 peserta asuransi tinggal di daerah southeast.

-->

In [ ]:
# your code here


___
4. Berdasarkan data nomor 3, berapa peserta yang aktif merokok?

- [ ] 1063 peserta
- [ ] 364 peserta
- [ ] 274 peserta

<!-- 

Sejumlah 1063 dari 1337 peserta asuransi bukanlah perokok. Jadi yang aktif merokok sebanyak 274 peserta.

-->

### 3. Data Preprocessing

Data preprocessing merupakan proses untuk membersihkan data dari data mentah menjadi data yang baik dan lengkap untuk proses modeling. Data yang bersih dan lengkap akan menghasilkan model machine learning yang efektif dan efisien serta memiliki performa yang lebih baik.

Proses ini bertujuan untuk mengurangi waktu komputasi dan resource yang dibutuhkan saat proses pembuatan model dan meningkatkan performa model machine learning.

Tahapan-tahapan yang dapat dilakukan pada data preprocessing antara lain:

> - Menangani data outliers
> - Menangani data kategorikal
> - Membagi dataset menjadi data train dan test
> - Melakukan scaling data

#### 3.1 Handling Numerical Data: Outliers

Outlier adalah observasi dengan nilai yang jauh berbeda dari mayoritas data. Hal ini dapat disebabkan oleh kesalahan input, anomali, atau kesalahan perhitungan. Menangani outlier penting dalam pemodelan machine learning karena dapat memengaruhi performa model.

 🔎 **Menangani Outliers**
 
Ada beberapa cara untuk menangani outliers:
1. **Remove outlier**: Menghilangkan dan menghapus keseluruhan outlier. Metode ini tidak disarankan jika jumlah outliers > 5% dari total data.
2. **Imputasi**: Mengganti nilai outliers dengan nilai yang diperbolehkan (sesuai kebutuhan).
3. ***Scaling***: Teknik untuk mentrasformasi sehingga nilai-nilai outlier ini tidak akan berpengaruh besar terhadap hasil analisis. Teknik scaling bermacam-macam, seperti standarisasi, normalisasi, min-max scaler dan lain sebagainya.


Mari kita identifikasi nilai outlier pada data kita dengan menggunakan boxplot:

#### Quiz 2

___
1. Menggunakan menggunakan boxplot pada kolom numerik, kolom mana yang datanya paling tersebar?

> Hint : Anda harus mendefinisikan kolom mana saja yang bertipe numerik (selain target) lalu buat boxplot untuk kolom tersebut.

- [ ] Age
- [ ] BMI
- [ ] Children

<!-- 

Reference answer :

```python
# list kolom numerik
num_features = ['age', 'bmi', 'children']

# visualisasi boxplot
insurance.boxplot(column=num_features)
plt.show()
```

Kolom age memiliki box yang paling lebar di antara kolom bmi dan children. Hal ini menandakan bahwa kolom age memiliki data yang paling tersebar (ditandai dengan nilai standar deviasi yang paling tinggi di antara kolom bmi dan children).

-->


In [ ]:
# your code here


> 💡 **INSIGHT:** Dari visualisasi boxplot di atas terlihat adanya outlier pada kolom `bmi` dan terlihat sedikit. Mari kita lakukan observasi lanjutan untuk mengetahui jumlah outlier dan apakah memungkinkan untuk dihapus atau tidak.

In [ ]:
# cek total data sebelum penghapusan outlier
insurance.shape

In [ ]:
# Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
Q1 = insurance['bmi'].quantile(0.25)
Q3 = insurance['bmi'].quantile(0.75)

___
2. Menggunakan metode IQR, berapa batas atas untuk kolom bmi?

> Hint : Anda mungkin perlu menghitung nilai IQR dan menggunakan formula = Q3 + 1.5 * IQR,

- [ ] 48.05
- [ ] 47.55
- [ ] 47.31

<!-- 

Reference answer :

```python
# Hitung nilai IQR
IQR = Q3 - Q1

# Mendefinisikan batas bawah dan atas
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Melihat nilai batas atas
print(upper_bound)
```

Dengan menggunakan formula Q3 + 1.5 * IQR, maka batas atas kolom bmi yaitu 47.31

-->


In [ ]:
# your code here


In [ ]:
# menghitung jumlah outlier pada kolom bmi
outlier_bmi = (insurance['bmi'] > upper_bound).sum()

# menampilkan jumlah outlier pada kolom bmi
print(f"Outlier kolom BMI: {outlier_bmi}")

> Karena jumlah outlier sedikit, maka kita bisa gunakan metode yang pertama yaitu menghapus outlier.

In [ ]:
# mempertahankan data yang memiliki nilai diantara batas atas dan batas bawah
kondisi_1 = (insurance['bmi'] >= lower_bound)
kondisi_2 = (insurance['bmi'] <= upper_bound)

# lakukan subsetting berdasarkan 2 kondisi di atas
insurance = insurance[kondisi_1 & kondisi_2]

# melihat data setelah dihapus outlier
insurance

> Setelah outlier dihapus, jumlah data berkurang dari yang semula 1.337 data menjadi 1.328 data. Ada 9 data yang dihapus (outlier).

#### 3.2 Handling Categorical Data

Model machine learning hanya bisa belajar dari data berupa angka. Jadi jika ada prediktor yang memilili tipe data bukan angka, perlu diubah terlebih dahulu ke bentuk angka. 

Data kategorik dibagi menjadi 2 jenis yaitu **nominal** (tidak memiliki urutan) dan **ordinal** (memiliki urutan). Masin-masing jenis memiliki perlakukan yang berbeda dalam menanganinya.

- **Nominal** $\rightarrow$ menggunakan *dummy variable encoding* dengan fungsi `pd.get_dummies()`.
- **Ordinal** $\rightarrow$ menggunakan *ordinal encoding* dengan fungsi `.cat.rename_categories()`.

In [ ]:
# cek data
insurance.head()

Kolom kategorikal:
- Nominal $\rightarrow$ Sex, smoker, region.
- Ordinal $\rightarrow$ Tidak ada.

Karena tipe ordinal tidak ada, mari kita gunakan metode *dummy varible encoding* untuk tipe nominal.

Untuk melakukan **Dummy Variable Encoding** kita menggunakan `pd.get_dummies()` dengan memasukkan kolom yang ingin kita ubah. Adapun parameter antara lain:

- `data`: data yang ingin diubah menjadi numerikal

- `columns`: list nama kolom yang akan dilakukan dummy variable encoding

- `drop_first`: Drop salah satu kolom (kolom pertama). Default False (Tidak ada kolom di drop). True agar kolom hasil dummies tidak rendundan.

- `dtype` : Mengubah tipe data hasil encoding menjadi suatu tipe data

Dokumentasi `pd.get_dummies()` dapat diakses [di sini](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

Mari kita coba menerapkan **Dummy Variable Encoding** untuk kolom yang telah kita disepakati

In [ ]:
# list prediktor yang memiliki tipe nominal
nominal_features = ['sex', 'smoker', 'region']

# implementasi pd.get_dummies
insurance = pd.get_dummies(insurance, columns = nominal_features, drop_first=True, dtype='int64')
insurance.head()

#### 3.3 Train-Test Splitting (*Cross Validation*)

Untuk memahami performa model, membagi dataset menjadi training data dan testing data adalah strategi yang baik dengan menggunakan fungsi `train_test_split()`. Metode ini dikenal dengan istilah *Cross Validation* yang digunakan untuk mengetahui kemampuan model memprediksi data baru.

Pisahkan dataset dengan menggunakan fungsi `train_test_split()`. Anda harus melewati 3 parameter `features`, `target`, dan `test_size`. Selain itu, Anda dapat menggunakan `random_state` untuk memilih record secara acak.

Kita dapat menggunakan `train_test_split` dengan beberapa parameter sebagai berikut.
- `*arrays`: dataframe yang kita gunakan (dipisah , untuk yang prediktor dan target variable)
- `test_size`: jumlah persentase dari data yang akan digunakan sebagai data test
- `train_size`: jumlah persentase dari data yang akan digunakan sebagai data test (akan otomatis terisi jika `test_size` diberi nilai)
- `random_state`: nilai random number generator (RNG). Jika kita memasukkan suatu nilai integer untuk parameter ini maka akan menghasilkan hasil yang sama untuk nilai yang sama. Jika kita mengubah nilainya, maka hasilnya akan berbeda.

In [ ]:
# memilih kolom target
target = 'charges'

# menyimpan data kolom target ke dalam variabel y
y = insurance[target]

# menghapus kolom target sehingga tersisa kolom prediktor dan dimasukkan ke variabel X
X = insurance.drop(columns=[target])

In [ ]:
# split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.20,
                                                    random_state = 100)

#### Quiz 3

___
1. Berdasarkan proses `train_test_split`, berapa % data yang digunakan untuk menguji model?

- [ ] 20
- [ ] 70
- [ ] 80

<!-- 

Dataset insurance dibagi menjadi data train dan data test dengan proporsi 80% data train dan 20% data test. Jawaban = 20%

-->

___
2. Data apa saja yang digunakan untuk proses `train_test_split`?

- [ ] X dan y
- [ ] X_train dan y_train
- [ ] X_test dan y_test

<!-- 

Untuk proses splitting dataset, digunakan data dari variabel X yang berisi prediktor dan y yang berisi data target. X_train, X_test, y_train, dan y_test merupakan hasil setelah split dataset.

-->

In [ ]:
# cek dimensi data train
X_train.shape

In [ ]:
# cek dimensi data test
X_test.shape

#### 3.4 Scaling (*Z Score Nomalization*)

Proses scaling dilakukan untuk menyeragamkan skala data antar prediktor. Hal ini penting dilakukan karena jika skala data antar prediktor tidak seragam, maka model machine learning akan bias (lebih memperhatikan) prediktor yang memiliki skala yang lebih besar. Hal ini akan membuat prediktor lain yang memiliki skala lebih kecil seakan-akan tidak memiliki pengaruh terhadap model dan akan membuat performa model tidak maksimal.

Metode scaling sangat banyak sekali seperti standarisasi dan normalisasi. Untuk kasus ini kita akan menggunakan metode standarisasi Z score menggunakan fungsi `StandardScaler` dari library `sklearn` yang membuat data kita memiliki nilai `mean = 0` dan `standar deviasi = 1`.

#### Quiz 4

___
1. Proses fit scaling berlaku untuk data apa saja?

- [ ] X_train dan y_train
- [ ] X_train
- [ ] X_test

<!-- 

Reference answer :

```python
# membuat object dari class StandardScaler()
scaler = StandardScaler()

# proses fit scaling
scaler.fit(X_train[num_features])
```

Proses fit scaling hanya berlaku untuk data X_train. Hal ini dilakukan untuk mencegah kebocoran (leak) pada data test. Hal ini dilakukan agar informasi pada data test benar-benar tidak bocor/tidak diketahui saat pembuatan model sehingga hasil evaluasi model lebih dapat dipertanggung jawabkan.

-->

___
2. Proses transform scaling berlaku untuk data apa saja?

- [ ] X_train dan y_train
- [ ] X_test dan y_test
- [ ] X_train dan X_test

<!-- 

Reference answer :

```python
# proses transform scaling
X_train[num_features] = scaler.transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])
```

Proses transform diberlakukan untuk data X_train dan X_test. Data y_train dan y_test tidak dilakukan scaling karena merupakan kolom target.

-->

In [ ]:
# your code here


___
3. Setelah proses scaling, berapa kolom yang tersisa dari proses tersebut?

- [ ] 3
- [ ] 8
- [ ] 11

<!-- 

Reference answer :

```python
# melihat data setelah proses scaling
X_train.head()
```

Menggunakan kode di atas, terlihat bahwa setelah scaling, jumlah tetap 8. Karena proses scaling tidak menambah/mengurangi jumlah kolom. Hanya mentransformasi nilai menjadi skala yang lebih kecil.

-->

In [ ]:
# your code here


### 4. Model Building: Support Vector Regressor (SVR)

SVM walaupun lebih terkenal pada penggunaan case klasifikasi, namun sebenarnya SVM juga mampu diterapkan untuk kasus regresi dengan sangat baik (memprediksi nilai yang target variabelnya numerik). 

Secara simpel SVR akan menghasilkan *hyperplane* yang dapat mengikuti pola data dari target terhadap prediktor. *Hyperplane* ini yang digunakan untuk memprediksi nilai target pada data baru.

#### 4.1 SVR Implementation

🔻Setelah semua proses persiapan data selesai, kini adalah saatnya untuk 'mesin' belajar dan diuji oleh data kita.

Langkah-langkah membangun model SVM. 
> 1. Import library SVR dari SVM.
> 2. Buat variabel `model_svr` untuk menyimpan hasil model `SVR()`.

In [ ]:
# membuat object SVR dari class svm
model_svr = svm.SVR()

# proses training
model_svr.fit(X_train, y_train)

#### 4.2 Model Prediction

Setelah model dilatih, model siap digunakan untuk memprediksi data baru. Model akan memprediksi data test yang telah kita siapkan sebelumnya.

In [ ]:
# prediksi data test
y_pred = model_svr.predict(X_test)

# menampilkan 10 data hasil prediksi
y_pred[:10]

**💡NOTES**: Walaupun kita sudah berhasil memprediksi, kita tetap perlu memastikan bahwa hasil prediksinya banyak yang benar / sudah sesuai dengan melakukan model evaluation

### 5. Model Evaluation

Setelah kita melakukan prediksi akan dilakukan evaluasi model dengan membandingkan nilai aktual dan hasil prediksi. Perbedaan antara nilai prediksi dengan nilai aktual disebut sebagai *error*, semakin kecil *error* maka semakin bagus model kita, karena nilai prediksi semakin mendekati nilai aslinya.

Salah satu cara yang sering digunakan dalam menghitung error adalah MAE ***(Mean Absolute Error)***. Yaitu dengan menghitung rata-rata error untuk semua prediksi.

Berikut adalah formula menghitung MAE:

$$MAE = \frac{\sum_{i=1}^{n}\left | y_{pred, i} - y_{real, i} \right |}{n}$$

Mean Absolute Error (MAE): Metriks untuk regresi yang menghitung rata-rata error untuk semua prediksi.

- Semakin kecil nilai MAE, semakin baik model kita
- Semakin besar nilai MAE, semakin buruk model kita.

🔻 Fungsi `mean_absolute_error()` memungkinkan kita Untuk menghitung *Mean Absolute Error*, kita dapat menggunkaan fungsi `mean_absolute_error` dari `sklearn.metrics`.

 Parameter yang dimasukkan adalah;
* `y_true` : data target aktual (*ground truth*) yang akan dijadikan acuan nilai sebenarnya
* `y_pred` : data target hasil prediksi model

Dokumentasi metrik evaluasi MAE dan metrik lainnya dapat diakses [di sini](https://scikit-learn.org/stable/api/sklearn.metrics.html).

> Untuk mengetahui apakah model yang Anda buat underfit, good fit, atau overfit, mari bandingkan peforma model ketika memprediksi data train dan data test.

#### Quiz 5

___
1. Berapa nilai MAE pada data train yang dihasilkan oleh model regresi?

> ✨ Hint: gunakan `model_svr` untuk memprediksi data train. Dengan menggunakan MAE, bandingkan nilai aktual pada data train dan hasil prediksi `model_svr`.

- [ ] 8,360.92
- [ ] 8,243.41
- [ ] 8,110.53

<!-- 
Reference answer :

```python
# prediksi data train
y_pred_train = model_svr.predict(X_train)
```

Berdasarkan kode di atas, didapat nilai MAE pada data train sebesar 8,243.41 (dalam format Bahasa Inggris)


-->


Prediksi model menggunakan data train:

In [ ]:
# your code here


In [ ]:
# evaluasi model
mae_train = metrics.mean_absolute_error(y_train, y_pred_train)

# tampilkan hasil evaluasi
print(f"MAE Train : {mae_train:,.2f}")

___
2. Berapa nilai MAE pada data test yang dihasilkan oleh model regresi?

> ✨ Hint: gunakan `model_svr` untuk memprediksi data test. Dengan menggunakan MAE, bandingkan nilai aktual pada data test dan hasil prediksi `model_svr`.

- [ ] 8,360.92
- [ ] 8,243.41
- [ ] 8,110.53

<!-- 
Reference answer :

```python
# prediksi data test
y_pred_test = model_svr.predict(X_test)
```

Berdasarkan kode di atas, didapat nilai MAE pada data test sebesar 8,360.92 (dalam format Bahasa Inggris)

-->


Prediksi model menggunakan data test:

In [ ]:
# your code here


In [ ]:
# evaluasi model
mae_test = metrics.mean_absolute_error(y_test, y_pred_test)

# tampilkan hasil evaluasi
print(f"MAE Test : {mae_test:,.2f}")

> ✨ NOTE: Untuk mengetahui apakah model sudah baik atau belum berdasarkan nilai MAE, Anda perlu membanginya dengan rentang nilai aslinya. Model regresi yang baik memiliki nilai MAE < 10%.

In [ ]:
# menghitung rentang nilai data train
range_train = y_train.max() - y_train.min()
mae_train_percent = round(mae_train / range_train * 100, 2)

# menghitung rentang nilai data test
range_test = y_test.max() - y_test.min()
mae_test_percent = round(mae_test / range_test * 100, 2)

In [ ]:
# menampilkan perbandingan evaluasi model pada data train dan data test
pd.DataFrame(data = {"MAE": [mae_train, mae_test],
                     "MAE (%)": [mae_train_percent, mae_test_percent]},
             index = ["Data Train", 'Data Test'])

> 💡 **INSIGHT:** Dari hasil evaluasi, dapat dikatakan bahwa model kita belum memiliki performa yang baik karena nilai MAE > 10%. Untuk memperbaiki performa, perlu kita lakukan proses tuning hyperparameter.

### 6. Model Improvement: Tuning Hyperparameter SVR (Opsional)

Seperti pada SVC, SVR juga memiliki *hyperparameter* yang dapat diatur untuk meningkatkan performa model.

Adapun *hyperparameter* yang sering digunakan dalam model SVR antara lain:
- `C (regularization) [default = 1, range = positive number]`: Sama seperti regularisasi pada SVC, semakin tinggi C pada SVR, maka membuat model semakin menyesuaikan data dengan lebih baik namun beresiko overfitting.
- `kernel [default = 'rbf', option = ['rbf', 'poly', 'linear', 'sigmoid']`: Mempengaruhi bagaimana garis regresi (*hyperplane*) terbentuk, untuk data yang dapat memiliki pola linear (garis lurus) dapat menggunakan 'linear'. Untuk data-data dengan pola non-linear dapat menggunakan kernel `rbf`.
*Note: Kebanyakan data berbentuk non-linear, sehingga sebaiknya selalu memilih kernel `rbf`*
- `epsilon [default = 0.1, range = 0.0 - inf]` : Epsilon mengatur tingkat kesalahan yang ditoleransi oleh model. Semakin kecil nilai epsilon, maka semakin ketat batas toleransi oleh model, sehingga model lebih kompleks. Begitupun sebaliknya.

Untuk hyperparameter dan penjelasan lebih detail dapat dilihat pada [dokumentasi SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

Mari kita coba untuk tuning model regresi kita dengan nama `model_svr_tuning`

In [ ]:
# membuat model svc tuning
model_svr_tuning = svm.SVR(C = 1000, kernel = 'rbf', epsilon=0.1)

# melakukan pelatihan ulang
model_svr_tuning.fit(X_train, y_train)

Mari kita lihat performa model hasil tuning untuk memprediksi data train.

In [ ]:
# prediksi data train menggunakan model hasil tuning
y_pred_train_tuning = model_svr_tuning.predict(X_train)

# evaluasi model
mae_train_tuning = metrics.mean_absolute_error(y_train, y_pred_train_tuning)

Mari kita lihat performa model hasil tuning untuk memprediksi data test.

In [ ]:
# prediksi data test menggunakan model hasil tuning
y_pred_test_tuning = model_svr_tuning.predict(X_test)

# evaluasi model
mae_test_tuning = metrics.mean_absolute_error(y_test, y_pred_test_tuning)

In [ ]:
# menghitung rentang nilai data train
mae_train_tuning_percent = round(mae_train_tuning / range_train * 100, 2)

# menghitung rentang nilai data test
mae_test_tuning_percent = round(mae_test_tuning / range_test * 100, 2)

Mari kita lihat perbandingan performa model data data train dan test sebelum dan sesudah tuning.

In [ ]:
# Menampilkan perbandingan evaluasi model hasil tuning pada data train dan data test
pd.DataFrame(data = {"MAE Train": [mae_train, mae_test],
                     "MAE Train (%)": [mae_train_percent, mae_test_percent],
                     "MAE Test": [mae_train_tuning, mae_test_tuning],
                     "MAE Test (%)": [mae_train_tuning_percent, mae_test_tuning_percent]},
             index = ["Base Model", 'Tuning Model'])

**✨ INSIGHT ✨**
> Setelah dilakukan proses tuning, skor MAE menjadi 5%. Hal ini menandakan bahwa model hasil tuning sudah bisa dikatakan model yang baik. Model yang dihasilkan merupakan model yang **Good Fit** karena nilai MAE pada data train dan data test tidak jauh berbeda.